In [67]:
#Most of these will be removed before the final submission. They are holdovers from the code I used to make this.
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import xgboost as xgb

from graphviz import Source
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.neural_network import MLPClassifier
from sklearn import neural_network
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

import math
import csv
import os

In [68]:
#Get the current path where the code is saved
path = os.getcwd()

#Change working directory for Python to the location where the code is saved
os.chdir(path)

#Read the excel file with data. Make sure it is saved in the same location as the code!!
data = pd.read_excel("Airbnb_Open_Data.xlsx")

#Give a description of the data that was loaded
print("\nHead:\n",data.head())
print("\nDescription:\n",data.price.describe())

#Create a dataframe with the new data,
frame=DataFrame(data)

#Assign columns 2-18 as variable X. X will represent the independent variables we will use to make predictions.
X = data.iloc[:,1:17]


Head:
                                                NAME host_identity_verified  \
0                Clean & quiet apt home by the park            unconfirmed   
1                             Skylit Midtown Castle               verified   
2               THE VILLAGE OF HARLEM....NEW YORK !            unconfirmed   
3  Entire Apt: Spacious Studio/Loft by central park               verified   
4         Large Cozy 1 BR Apartment In Midtown East               verified   

  neighbourhood group neighbourhood  instant_bookable cancellation_policy  \
0            Brooklyn    Kensington             False              strict   
1           Manhattan       Midtown             False            moderate   
2           Manhattan        Harlem              True            flexible   
3           Manhattan   East Harlem             False            moderate   
4           Manhattan   Murray Hill              True            flexible   

         room type  Construction year  price  service fee  m

In [4]:
#If un-commented, this statement can be used to show how many columns are in X.
#print("\n\nX: \n",len(X.iloc[0,:]))

In [69]:
#Assign the "price" variable to y. y will represent the dependent variable for which we are solving.
y = X.price

#Remove columns that will not be used in this analysis. We need to drop "price" and anything having to do with reviews. 
X = X.drop(columns=['last review', 'reviews per month', 'number of reviews', 'review rate number', 'price'])

#Create dummy variables for columns with character data types
X = pd.get_dummies(X, columns=['host_identity_verified', 'neighbourhood group', 'neighbourhood', 'instant_bookable', 'cancellation_policy', 'room type'])

#Show details about the dataset after adding dummy variables
X.head()

,Construction year,service fee,minimum nights,calculated host listings count,availability 365,host_identity_verified_unconfirmed,host_identity_verified_verified,neighbourhood group_Bronx,neighbourhood group_Brooklyn,neighbourhood group_Manhattan,...,neighbourhood_Woodside,instant_bookable_False,instant_bookable_True,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,room type_Entire home/apt,room type_Hotel room,room type_Private room,room type_Shared room
0,2020.0,193.0,10.0,6.0,286.0,1,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0
1,2007.0,28.0,30.0,2.0,228.0,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,0
2,2005.0,124.0,3.0,1.0,352.0,1,0,0,0,1,...,0,0,1,1,0,0,0,0,1,0
3,2009.0,41.0,10.0,1.0,289.0,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,0
4,2013.0,115.0,3.0,1.0,374.0,0,1,0,0,1,...,0,0,1,1,0,0,1,0,0,0


In [70]:
#Split the data into test and training data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=34)

In [7]:
#If un-commented, these statements can be used to confirm that the X and y sets created are the correct length. This is useful in troubleshooting.
#print(len(X_train))
#print(len(X_train))
#print(len(X_test))
#print(len(y_test))

In [71]:
#Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
    
#Initialize Lasso model
lasso = Lasso(random_state=34)

#Define a range of alpha values to search
param_grid = {'alpha': np.logspace(-4, 0, 50)}

#Use GridSearchCV to find the optimal alpha using cross-validation
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

#Get the best Lasso model
best_lasso = grid_search.best_estimator_
print(f"Best alpha: {best_lasso.alpha}")

#Access the coefficients
coefficients = best_lasso.coef_
feature_names = X.columns

#Identify selected features (non-zero coefficients)
selected_features = feature_names[coefficients != 0]
discarded_features = feature_names[coefficients == 0]

#Print details about the features and coefficients that were selected
print(f"\n\nSelected Features: {list(selected_features)}")
#print(f"\n\nDiscarded Features: {list(discarded_features)}")
print(f"\n\nCoefficients: {dict(zip(feature_names, coefficients))}")

#Evaluate the model
y_pred = best_lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"\nMean Squared Error on Test Set: {mse}")

Best alpha: 0.009102981779915217


Selected Features: ['Construction year', 'service fee', 'neighbourhood_Allerton', 'neighbourhood_Arverne', 'neighbourhood_Belmont', 'neighbourhood_Borough Park', 'neighbourhood_Breezy Point', 'neighbourhood_Bronxdale', 'neighbourhood_Cambria Heights', 'neighbourhood_Co-op City', 'neighbourhood_Concord', 'neighbourhood_Dyker Heights', 'neighbourhood_East Flatbush', 'neighbourhood_East Village', 'neighbourhood_Emerson Hill', 'neighbourhood_Fordham', 'neighbourhood_Fort Greene', 'neighbourhood_Glendale', 'neighbourhood_Gowanus', 'neighbourhood_Hollis', 'neighbourhood_Huguenot', 'neighbourhood_Jackson Heights', 'neighbourhood_Kips Bay', 'neighbourhood_Little Neck', 'neighbourhood_Manhattan Beach', 'neighbourhood_Oakwood', 'neighbourhood_Parkchester', 'neighbourhood_Port Morris', 'neighbourhood_Queens Village', 'neighbourhood_Rego Park', 'neighbourhood_Roosevelt Island', 'neighbourhood_Rossville', 'neighbourhood_Soundview', 'neighbourhood_South Beach', 'ne

In [44]:
#print("\nX:\n",X.columns)

In [72]:
print("\nIndependent variable training set length before feature selection:\n",len(X_train.columns))
print("\nIndependent variable test set length before feature selection:\n",len(X_test.columns))



X_new = X.drop(columns=discarded_features)

#Split the data into test and training data
X_train, X_test, y_train, y_test = train_test_split(X_new,y, test_size=0.3, random_state=34)

print("\n\nIndependent variable training set length after feature selection:\n",len(X_train.columns))
print("\nIndependent variable test set length after feature selection:\n",len(X_test.columns))



Independent variable training set length before feature selection:
 246

Independent variable test set length before feature selection:
 246


Independent variable training set length after feature selection:
 41

Independent variable test set length after feature selection:
 41


In [47]:
#Create a linear regression model for the data
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [48]:
#Determine accuracies for the predictions
accuracies = cross_val_score(lr, X_train, y_train, cv=10)
print("\n\nAccuracies:\n", accuracies)



Accuracies:
 [0.99998182 0.99998161 0.9999816  0.99998199 0.99998189 0.99998174
 0.99998172 0.99998174 0.999982   0.99998177]


In [49]:
#Fit the linear regression to the training model
lr.fit(X_train, y_train)

#Print the coefficients and intercept
print("\nLR Coef:",lr.coef_)
#print("\n\nMAX", lr.coef_.max())
print("\nLR Intercept:",lr.intercept_)

#Use the model to make predictions on the test and training sets
lr_train_pred = lr.predict(X_train)
lr_test_pred = lr.predict(X_test)


LR Coef: [-1.82871827e-03  4.99965069e+00 -3.87874754e-01  2.95321933e-01
  5.67461939e-01 -2.17313673e-01  1.12357729e+00 -8.13976546e-01
  4.16527821e-01 -1.25070428e+00  3.95310201e-01 -8.82002103e-01
 -8.31965319e-02  5.49194384e-02  1.11930952e+00 -2.75571091e-01
  1.17941373e-01  2.56311015e-01  2.15225933e-01  4.74760488e-01
 -9.85514397e-01 -1.59518860e-01  1.02482732e-01  1.20535683e+00
  7.17818189e-01 -7.81976976e-01  3.31563271e-01 -2.81331977e-01
 -4.11983336e-01 -2.77703534e-01 -2.43280227e-01  1.96220049e+00
  6.38419956e-01 -8.11905201e-01 -2.71569801e-01 -2.94279109e-01
  3.87969976e-01  2.98861390e-01  4.83537152e-01  2.26885802e-02
 -2.26281542e-02]

LR Intercept: 3.7232640835768507


In [50]:
#Use cross-validation to determine the accuracy of the linear regression
scoring = ['accuracy', 'precision_macro', 'recall_macro']
scores = cross_validate(lr, X_train, y_train, scoring='r2', cv=10, return_train_score = True)
scores_frame=pd.DataFrame(scores)
#print("\n\nScores:\n",scores_frame)

#Print metrics for the test and training sets
print(metrics.r2_score(y_train, lr_train_pred))
print(metrics.mean_squared_error(y_train, lr_train_pred))
print(metrics.r2_score(y_test, lr_test_pred))
print(metrics.mean_squared_error(y_test, lr_test_pred))

0.9999818138087907
2.00339184009219
0.9999817301138059
2.0007145555897807


In [18]:
#Create a decision tree model
RT = tree.DecisionTreeRegressor(min_samples_leaf=5, random_state=39)

#Fit the model to the training data
RT.fit(X_train, y_train)
RT_pred_train = RT.predict(X_train)
RT_pred_test = RT.predict(X_test)
print("\nRegression Tree Training Set Mean Squared Error:",metrics.mean_squared_error(y_train, RT_pred_train))
print("\nRegression Tree Training Set R2:\n",metrics.r2_score(y_train, RT_pred_train))
print("\nRegression Tree Test Set Mean Squared Error:",metrics.mean_squared_error(y_test, RT_pred_test))
print("\nRegression Tree Test Set R2:\n",metrics.r2_score(y_test, RT_pred_test))

RT.feature_importances_
pd.DataFrame({'variable':X_final.columns[:247], 'importance':RT.feature_importances_})
dot_data = tree.export_graphviz(RT, out_file=None, feature_names = X_train.columns)
Source(dot_data).render('Regression_Tree')

scoring = ['accuracy', 'precision_macro', 'recall_macro']
scores = cross_validate(RT, X_train, y_train, scoring='r2', cv=10, return_train_score = True)
scores_frame=pd.DataFrame(scores)
print("\n\nScores:\n",scores_frame)



Regression Tree Training Set Mean Squared Error: 0.8826185581769392

Regression Tree Training Set R2:
 0.9999919878530287

Regression Tree Test Set Mean Squared Error: 2.4180215899124122

Regression Tree Test Set R2:
 0.9999779193992769


Scores:
    fit_time  score_time  test_score  train_score
0  0.951659    0.008801    0.999977     0.999992
1  0.879055    0.006331    0.999977     0.999992
2  0.887552    0.006376    0.999978     0.999992
3  0.889791    0.006389    0.999978     0.999992
4  0.883221    0.006214    0.999978     0.999992
5  0.879890    0.006327    0.999977     0.999992
6  0.878751    0.006287    0.999977     0.999992
7  0.959265    0.006234    0.999978     0.999992
8  0.899534    0.006300    0.999977     0.999992
9  0.891109    0.006349    0.999978     0.999992


In [53]:
#Random forest for regression
RF = RandomForestRegressor(n_estimators=100, random_state=34)
RF.fit(X_train, y_train)

RF_pred_test = RF.predict(X_test)

scoring = ['accuracy', 'precision_macro', 'recall_macro']
scores = cross_validate(RF, X_train, y_train, scoring='r2', cv=10, return_train_score = True)
scores_frame=pd.DataFrame(scores)
print("\n\nScores:\n",scores_frame)



Scores:
    fit_time  score_time  test_score  train_score
0  8.719561    0.141418    0.999979     0.999989
1  8.659681    0.140445    0.999979     0.999989
2  8.491918    0.146085    0.999979     0.999989
3  8.496411    0.140506    0.999980     0.999989
4  8.655907    0.139992    0.999979     0.999989
5  8.642315    0.140292    0.999979     0.999989
6  8.621330    0.140595    0.999979     0.999989
7  8.615679    0.140573    0.999979     0.999989
8  8.599185    0.140271    0.999980     0.999989
9  8.629322    0.140892    0.999979     0.999989


In [74]:
#Initialize and train an XGBoost classifier
xg_boost = xgb.XGBRegressor(objective='reg:squarederror',  # Objective function for regression
                         n_estimators=100,             # Number of boosting rounds
                         learning_rate=0.1,            # Step size shrinkage
                         max_depth=5,                  # Maximum depth of a tree
                         random_state=42)
xg_boost.fit(X_train, y_train)

#Make predictions
xgb_pred_test = xg_boost.predict(X_test)

scoring = ['accuracy', 'precision_macro', 'recall_macro']
scores = cross_validate(xg_boost, X_train, y_train, scoring='r2', cv=10, return_train_score = True)
scores_frame=pd.DataFrame(scores)
print("\n\nScores:\n",scores_frame)



Scores:
    fit_time  score_time  test_score  train_score
0  0.381207    0.016145    0.999982     0.999982
1  0.382325    0.014797    0.999982     0.999982
2  0.351843    0.014526    0.999982     0.999982
3  0.352255    0.014444    0.999982     0.999982
4  0.348048    0.014466    0.999982     0.999982
5  0.395431    0.015138    0.999982     0.999982
6  0.343884    0.014276    0.999982     0.999982
7  0.347569    0.014306    0.999982     0.999982
8  0.364444    0.014447    0.999982     0.999982
9  0.353673    0.014421    0.999982     0.999982
